In [1]:
import util.utils as utils

percentage = 0.02
rand_seed = 42
data_sampling_freq = '10s'
rolling_window_size = '10s'

df_result = utils.get_binary(percentage, rand_seed)
df_face = utils.get_face(percentage, rand_seed, data_sampling_freq, rolling_window_size)

display(df_face)
display(df_result)

CLNFgaze_x_0  CLNFgaze_y_0  CLNFgaze_z_0  CLNFgaze_x_1  \
ID  TIMESTAMP                                                                 
386 0 days 00:00:00      0.020882      0.114932     -0.985159     -0.187369   
    0 days 00:00:10      0.060876      0.158891     -0.979911     -0.185456   
    0 days 00:00:20      0.048942      0.159756     -0.984069     -0.185223   
    0 days 00:00:30      0.051738      0.165190     -0.983668     -0.174363   
    0 days 00:00:40      0.049782      0.190049     -0.979635     -0.177583   
...                           ...           ...           ...           ...   
391 0 days 00:10:40     -0.000396      0.162813     -0.982788     -0.172738   
    0 days 00:10:50     -0.002969      0.168443     -0.975233     -0.173063   
    0 days 00:11:00     -0.008404      0.096919     -0.986678     -0.171271   
    0 days 00:11:10     -0.009584      0.035722     -0.991693     -0.076398   
    0 days 00:11:20      0.000000      0.000000     -1.000000      0.000000   

                     CLNFgaze_y_1  CLNFgaze_z_1  CLNFgaze_x_h0  CLNFgaze_y_h0  \
ID  TIMESTAMP                                                                   
386 0 days 00:00:00      0.090528     -0.967401       0.113978       0.211860   
    0 days 00:00:10      0.139311     -0.964433       0.013874       0.204188   
    0 days 00:00:20      0.131315     -0.971070       0.017197       0.242405   
    0 days 00:00:30      0.125572     -0.974557       0.018452       0.232494   
    0 days 00:00:40      0.144044     -0.972055       0.019017       0.253643   
...                           ...           ...            ...            ...   
391 0 days 00:10:40      0.182752     -0.962403       0.132738       0.410371   
    0 days 00:10:50      0.101383     -0.971495       0.400424       0.394142   
    0 days 00:11:00      0.141179     -0.946717       0.234962       0.192641   
    0 days 00:11:10      0.035174     -0.982470       0.060823       0.138469   
    0 days 00:11:20      0.000000     -1.000000       0.000000       0.000000   

                     CLNFgaze_z_h0  CLNFgaze_x_h1  ...  CLNFfeatures3D_Z58  \
ID  TIMESTAMP                                      ...                       
386 0 days 00:00:00      -0.946309      -0.091666  ...          519.613237   
    0 days 00:00:10      -0.971987      -0.230672  ...          540.856410   
    0 days 00:00:20      -0.964628      -0.215217  ...          519.459970   
    0 days 00:00:30      -0.970581      -0.206811  ...          517.561633   
    0 days 00:00:40      -0.966128      -0.207692  ...          520.462397   
...                            ...            ...  ...                 ...   
391 0 days 00:10:40      -0.886184      -0.039702  ...          596.082983   
    0 days 00:10:50      -0.777415       0.270857  ...          611.819750   
    0 days 00:11:00      -0.824262       0.089705  ...          549.835507   
    0 days 00:11:10      -0.941188      -0.003228  ...          642.796860   
    0 days 00:11:20      -1.000000       0.000000  ...          468.863000   

                     CLNFfeatures3D_Z59  CLNFfeatures3D_Z60  \
ID  TIMESTAMP                                                 
386 0 days 00:00:00          523.152257          528.202373   
    0 days 00:00:10          546.058773          553.371500   
    0 days 00:00:20          525.978237          534.036180   
    0 days 00:00:30          524.068173          531.824040   
    0 days 00:00:40          527.595803          536.371060   
...                                 ...                 ...   
391 0 days 00:10:40          600.297053          605.735340   
    0 days 00:10:50          611.001257          612.771850   
    0 days 00:11:00          551.907377          558.196050   
    0 days 00:11:10          642.991900          645.226560   
    0 days 00:11:20          466.043000          460.653000   

                     CLNFfeatures3D_Z61  CLNFfeatures3D_Z62  \
ID  TIMESTAMP                            

,PHQ_Binary
ID,
386,1
391,0


### EDA
* eigenfaces
* distribution of landmarks / action units

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

# Set up the plotting style
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 8]

# Extract landmark features (x, y coordinates)
landmark_features = [col for col in df_face.columns if 'CLNFfeatures_' in col]
landmark_data = df_face[landmark_features]

# Perform PCA for eigenfaces
pca = PCA(n_components=10)
landmark_pca = pca.fit_transform(landmark_data)

# Plot explained variance ratio
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Explained Variance Ratio of Landmark PCA Components')
plt.show()

# Plot first few eigenfaces
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i in range(6):
    if i < len(pca.components_):
        component = pca.components_[i].reshape(-1, 2)  # Reshape to (68, 2) for x,y coordinates
        axes[i].scatter(component[:, 0], component[:, 1], alpha=0.6)
        axes[i].set_title(f'Eigenface {i+1}')
        axes[i].set_xlabel('X coordinate')
        axes[i].set_ylabel('Y coordinate')

plt.tight_layout()
plt.show()

# Distribution of Action Units
au_features = [col for col in df_face.columns if 'CLNFAUs_' in col]

# Create violin plots for Action Units
plt.figure(figsize=(15, 8))
sns.violinplot(data=df_face[au_features])
plt.xticks(rotation=45, ha='right')
plt.title('Distribution of Action Units')
plt.tight_layout()
plt.show()

# Correlation matrix of Action Units
plt.figure(figsize=(12, 10))
sns.heatmap(df_face[au_features].corr(), cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Action Units')
plt.tight_layout()
plt.show()

# Distribution of landmark positions
# Take mean positions for each ID to get a general idea of face shape
mean_landmarks = df_face.groupby('ID')[landmark_features].mean()

x_coords = mean_landmarks[[col for col in landmark_features if '_x' in col]].values
y_coords = mean_landmarks[[col for col in landmark_features if '_y' in col]].values

plt.figure(figsize=(10, 10))
plt.scatter(x_coords, y_coords, alpha=0.5)
plt.title('Average Landmark Positions')
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.axis('equal')
plt.show()


### PCA

In [ ]:
# Extract different feature types
gaze_features = [col for col in df_face.columns if 'CLNFgaze_' in col]
au_features = [col for col in df_face.columns if 'CLNFAUs_' in col]
pose_features = [col for col in df_face.columns if 'CLNFpose_' in col]

# Dictionary to store PCA results
pca_results = {}

# Perform PCA for each feature type
for feature_name, features in [('Gaze', gaze_features), 
                             ('Action Units', au_features),
                             ('Pose', pose_features)]:
    
    # Skip if no features found
    if not features:
        continue
        
    # Extract data
    feature_data = df_face[features]
    
    # Perform PCA
    pca = PCA(n_components=min(10, len(features)))
    feature_pca = pca.fit_transform(feature_data)
    
    # Store results
    pca_results[feature_name] = {
        'pca': pca,
        'transformed': feature_pca
    }
    
    # Plot explained variance ratio
    plt.figure(figsize=(10, 6))
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance Ratio')
    plt.title(f'Explained Variance Ratio of {feature_name} PCA Components')
    plt.grid(True)
    plt.show()
    
    # Print variance explained by each component
    print(f"\n{feature_name} - Explained variance ratio by component:")
    for i, ratio in enumerate(pca.explained_variance_ratio_):
        print(f"PC{i+1}: {ratio:.4f}")
    
    # Plot feature importance heatmap
    plt.figure(figsize=(12, 8))
    feature_importance = pd.DataFrame(
        pca.components_.T,
        columns=[f'PC{i+1}' for i in range(feature_pca.shape[1])],
        index=features
    )
    sns.heatmap(feature_importance, cmap='coolwarm', center=0)
    plt.title(f'Feature Importance in {feature_name} Principal Components')
    plt.tight_layout()
    plt.show()

# Create combined PCA features DataFrame
pca_df = pd.DataFrame(index=df_face.index)
for feature_name, result in pca_results.items():
    feature_cols = [f'{feature_name}_PC{i+1}' for i in range(result['transformed'].shape[1])]
    pca_df[feature_cols] = result['transformed']

# Save the transformed features for later use
pca_features = pca_df.copy()

### train test split

### pipeline & hyperparameter tuning

### evaluation

In [2]:
# from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
# 
# y_pred = grid_search.predict(X_test)
# 
# y_test = y_test.to_numpy()
# # Debug: Check the shapes and types of y_pred
# print(f'y_pred: {y_pred}')
# print(f'y_test: {y_test}')
# 
# # Generate and display classification report
# print("Classification Report:")
# print(classification_report(y_test, y_pred))
# 
# # Generate and display confusion matrix
# cm = confusion_matrix(y_test, y_pred, labels=grid_search.classes_)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=grid_search.classes_)
# disp.plot(cmap=plt.cm.Blues)
# plt.show()
# 
